In [1]:
# Read API credentials from .env file

import os
import pandas as pd

client_id = os.environ['CLIENT_ID']
client_secret = os.environ['CLIENT_SECRET']
user_agent = os.environ['USER_AGENT']
username = os.environ['USER_NAME']
password = os.environ['PASSWORD']

In [59]:
import csv

def create_dict_posts(posts):
    result = {}
    posts_list = []
    for post in posts:
        result["Title"] = post.title
        result["Score"] = post.score
        result["Autor"] = post.author
        result["Time"] = post.created
        result["Comments"] = post.num_comments
        result["UpvoteRatio"] = post.upvote_ratio
        result["PostId"] = post.id
        result["Subreddit"] = post.subreddit.display_name
        result["is_nsfw"] = post.over_18
        result["SelfText"] = post.selftext
        result["IsSelfText"] = post.is_self
        result["IsPinned"] = post.pinned
        result["IsStickied"] = post.stickied
        result["IsPinned"] = post.pinned
        result["IsLocked"] = post.locked
        posts_list.append(result)
        result = {}
    return posts_list

def create_dict_posts_top(posts):
    result = {}
    posts_list = []
    for post in posts.top(limit=None):
        result["Title"] = post.title
        result["Score"] = post.score
        result["Autor"] = post.author
        result["Time"] = post.created
        result["Comments"] = post.num_comments
        result["UpvoteRatio"] = post.upvote_ratio
        result["PostId"] = post.id
        result["Subreddit"] = post.subreddit.display_name
        result["is_nsfw"] = post.over_18
        result["SelfText"] = post.selftext
        result["IsSelfText"] = post.is_self
        result["IsPinned"] = post.pinned
        result["IsStickied"] = post.stickied
        result["IsPinned"] = post.pinned
        result["IsLocked"] = post.locked
        posts_list.append(result)
        result = {}
    return posts_list

def store_in_CSV(file_name, posts):
    fields = ['Title', 'Score', 'Autor', 'Time', 'Comments', 'UpvoteRatio', 'PostId', 'Subreddit', 'is_nsfw', 'SelfText', 'IsSelfText', 'IsPinned', 'IsStickied', 'IsLocked']
    with open(file_name, 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fields)
        if csvfile.tell() == 0:
            writer.writeheader()
        writer.writerows(posts)

def scrap_dataset(subreddits, file_path, reddit):
    for sub in subreddits:
        search_results = reddit.subreddit(sub)
        posts = create_dict_posts_top(search_results)
        store_in_CSV(file_path, posts)
    df = pd.read_csv(file_path)
    print(df.shape)
    return df                          
    

In [26]:
import requests

In [15]:
auth = requests.auth.HTTPBasicAuth(client_id, client_secret)

In [16]:
data = {
    'grant_type': 'password',
    'username': username,
    'password': password
}
headers = {'User-Agent': user_agent}
res = requests.post('https://www.reddit.com/api/v1/access_token',
auth=auth, data=data, headers=headers)
print(res)

<Response [200]>


In [19]:
token = res.json()['access_token']
headers['Authorization'] = 'bearer {}'.format(token)
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)

<Response [200]>

In [54]:
api = 'https://oauth.reddit.com'
res = requests.get('{}/r/askscience/top?t=all'.format(api), headers=headers, params={'limit': '5000', 'after':None})
json_res = res.json()

In [63]:
def create_dict(data):
    result = {}
    posts_list = []
    for post in data:
        post_d = post['data']
        result["Title"] = post_d['title']
        result["Score"] = post_d['score']
        result["Autor"] = post_d.get('author_fullname', None)
        result["Time"] = post_d['created']
        result["Comments"] = post_d['num_comments']
        result["UpvoteRatio"] = post_d['upvote_ratio']
        result["Subreddit"] = post_d['subreddit']
        result["is_nsfw"] = post_d['over_18']
        result["SelfText"] = post_d['selftext']
        result["IsSelfText"] = post_d['is_self']
        result["IsPinned"] = post_d['pinned']
        result["IsStickied"] = post_d['stickied']
        result["IsPinned"] = post_d['pinned']
        result["IsLocked"] = post_d['locked']
        posts_list.append(result)
        result = {}
    return posts_list

In [67]:
after = None
counter = 0
while True:
    res = requests.get('{}/search/?q=working%20at%20amazon'.format(api), headers=headers, params={'limit': '5000', 'after': after})
    json_res = res.json()
    data = json_res['data']
    after = data['after']
    dic_data = create_dict(data['children'])
    store_in_CSV('wamazon.csv',dic_data)
    counter += 1
    if after == None or counter == 50:
        break
    

In [68]:
df = pd.read_csv('wamazon.csv')
df.shape

(221, 14)

In [69]:
df.head()

,Title,Score,Autor,Time,Comments,UpvoteRatio,PostId,Subreddit,is_nsfw,SelfText,IsSelfText,IsPinned,IsStickied,IsLocked
0,I can't fucking stand working at Amazon,345,t2_vy5bzfd0,1.679421e+09,173,0.91,NaN,antiwork,False,I have ADHD. I work at Amazon as a picker. I a...,True,False,False,False
1,I hate working at Amazon,37,t2_8oc0wkdx,1.677656e+09,42,0.76,NaN,AmazonFC,False,I'm not certain if it's just how things are go...,True,False,False,False
2,Anyone actually like working at Amazon?,55,t2_5ns0vqsb,1.635480e+09,130,0.89,NaN,AmazonFC,False,"Hello all,\n\nDoes anyone here like working at...",True,False,False,False
3,Is working at Amazon really as bad as people say?,31,t2_b59a00b6,1.619921e+09,107,0.90,NaN,AmazonFC,False,I am asking because I've been unemployed for a...,True,False,False,False
4,Is working at amazon worth it or nah?,3,t2_4gzf4cf6,1.675802e+09,51,0.57,NaN,AmazonFC,False,I don’t have a job and Im looking and obviousl...,True,False,False,False


In [53]:
len(json_res['data']['children'])

100